# Исследование надежности заемщиков для кредитного отдела банка

## 1. Описание проекта

**Цель: исследовать влияние семейного положения и количества детей клиента на факт погашения кредита в срок.** Получение данные будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заемщика вернуть кредит банку.

**Задачи исследования** — ответить на вопросы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?

**Данные для анализа:** данные о клиентах банка.

**План исследования:**
1. [Обзор данных.](#2.1)
2. Предобработка данных:
    * проверили [явные дубликаты](#3.1),
    * проверили [пропуски значений](#3.2),
    * оптимизировали [тип данных](#3.3),
    * проверили данные на [неявные дубликаты](#3.4).
3. Добавление данных:
    * категоризировали данные о [доходах](#4.1.1) и [целях кредита](#4.1.2),
    * создали словари с данными об [образовании](#4.2.1), [семейном положении](#4.2.2) и [доходе](#4.2.3).
4. Исследовательский анализ:
    * [зависимость между количеством детей и возвратом кредита в срок](#5.1),
    * [зависимость между семейным положением и возвратом кредита в срок](#5.2),
    * [зависимость между уровнем дохода и возвратом кредита в срок](#5.3),
    * [влияние целей кредита на его возврат в срок](#5.4).
5. [Итоги исследования.](#6)

## 2. Обзор данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
# функция для первичного анализа
def first_rev(df):
    print('info')
    display(df.info(memory_usage='deep'))
    print('describe')
    display(df.describe().T.round(3))
    print('head')
    display(df.head())
    print('duplicated')
    display(df.duplicated().sum())

### 2.1 Данные <a id="2.1"></a>

In [3]:
df = pd.read_csv(r"C:\Users\csvic\Desktop\data analysis\Проекты для git\2. Исследование надежности заемщиков для кредитного отдела банка.csv")

In [4]:
first_rev(df)

info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


None

describe


,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.539,1.382,-1.000,0.000,0.000,1.000,20.000
days_employed,19351.0,63046.498,140827.312,-18388.950,-2747.424,-1203.370,-291.096,401755.400
dob_years,21525.0,43.293,12.575,0.000,33.000,42.000,53.000,75.000
education_id,21525.0,0.817,0.548,0.000,1.000,1.000,1.000,4.000
family_status_id,21525.0,0.973,1.420,0.000,0.000,0.000,1.000,4.000
debt,21525.0,0.081,0.273,0.000,0.000,0.000,0.000,1.000
total_income,19351.0,167422.302,102971.566,20667.264,103053.153,145017.938,203435.068,2265604.029


head


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


duplicated


54

### 2.1 Вывод

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

**Дополнительно:**
* Удалить дубликаты.
* Проверить и заменить/удалить пропуски значений в колонках `days_employed` и `total_income`.
* Оптимизировать тип данных.
* Проверить датафрейм на неявные дубликаты.

## 3. Предобработка данных

### 3.1 Дубликаты <a id="3.1"></a>
Удалим явные дубликаты.

In [5]:
df = df.drop_duplicates().reset_index(drop=True)

In [6]:
df.duplicated().sum()

0

### 3.2 Пропуски значений <a id="3.2"></a>

In [7]:
df.isna().sum()

children               0
days_employed       2120
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2120
purpose                0
dtype: int64

In [8]:
df.isna().mean().round(2)

children            0.0
days_employed       0.1
dob_years           0.0
education           0.0
education_id        0.0
family_status       0.0
family_status_id    0.0
gender              0.0
income_type         0.0
debt                0.0
total_income        0.1
purpose             0.0
dtype: float64

In [9]:
df[
    df['days_employed'].isna()
    &df['total_income'].isna()
]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21435,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21441,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21443,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21448,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [10]:
df['days_employed'] = df['days_employed'].fillna(abs(df['days_employed']).median())
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [11]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### 3.2.1 Вывод

Люди с пропущенными значениями количества дней трудового стажа являются также людьми с пропущенным значением ежемесячного дохода (видно по количеству строк).

В выборке не просматривается какая-либо системность данных — разные количество детей, возраст, образование, семейное положение, гендер, тип занятости, наличие/отсутствие задолженности по возврату кредитов и цель получения кредита.

Пропуски в данных являются случайными. Это технологическая ошибка или люди по личной причине не захотели указывать свой доход.

**Заменили пропуски значений в `days_employed` и `total_income` на медианные значения.**

### 3.3 Тип данных <a id="3.3"></a>
Изменим тип данных для удобства и скорости анализа.

In [12]:
df['children'] = abs(df['children']).astype(np.uint8, errors='ignore')

In [13]:
columns_to_replace = [
    'dob_years',
    'education_id',
    'family_status_id',
    'debt'
]
for columns in columns_to_replace:
    df[columns] = df[columns].astype(np.uint8, errors='ignore')

In [14]:
df['days_employed'] = abs(df['days_employed']).astype(np.uint16, errors='ignore')

In [15]:
df['total_income'] = df['total_income'].astype(np.uint32, errors='ignore')

In [16]:
columns_to_replace = [
    'education',
    'family_status',
    'gender',
    'income_type',
    'purpose'
]
for columns in columns_to_replace:
    df[columns] = df[columns].astype('category')

In [17]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21471 entries, 0 to 21470
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   children          21471 non-null  uint8   
 1   days_employed     21471 non-null  uint16  
 2   dob_years         21471 non-null  uint8   
 3   education         21471 non-null  category
 4   education_id      21471 non-null  uint8   
 5   family_status     21471 non-null  category
 6   family_status_id  21471 non-null  uint8   
 7   gender            21471 non-null  category
 8   income_type       21471 non-null  category
 9   debt              21471 non-null  uint8   
 10  total_income      21471 non-null  uint32  
 11  purpose           21471 non-null  category
dtypes: category(5), uint16(1), uint32(1), uint8(5)
memory usage: 347.7 KB


#### 3.3.1 Вывод
Изменили тип данных и уменьшили размер используемой памяти с 13.5 MB до 347.7 KB.

### 3.4 Проверка на нормальность <a id="3.4"></a>
Проверим неявные дубликаты и выбивающиеся значения.

**Колонка `children`**

In [18]:
df['children'].value_counts()

0     14107
1      4856
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [19]:
df = df[df['children'] != 20].reset_index(drop=True)

**Колонка `education`**

In [20]:
df['education'].value_counts()

среднее                13653
высшее                  4698
СРЕДНЕЕ                  770
Среднее                  705
неоконченное высшее      666
ВЫСШЕЕ                   271
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [21]:
df['education'] = df['education'].str.lower()

In [22]:
df['education'].value_counts()

среднее                15128
высшее                  5237
неоконченное высшее      742
начальное                282
ученая степень             6
Name: education, dtype: int64

**Колонка `family_status`**

In [23]:
df['family_status'].value_counts()

женат / замужем          12295
гражданский брак          4151
Не женат / не замужем     2801
в разводе                 1193
вдовец / вдова             955
Name: family_status, dtype: int64

**Колонка `gender`**

In [24]:
df['gender'].value_counts()

F      14142
M       7252
XNA        1
Name: gender, dtype: int64

In [25]:
df = df[df['gender'] != 'XNA'].reset_index(drop=True)

**Колонка `income_type`**

In [26]:
df['income_type'].value_counts()

сотрудник          11048
компаньон           5057
пенсионер           3828
госслужащий         1455
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

**Колонка `debt`**

In [27]:
df['debt'].value_counts()

0    19661
1     1733
Name: debt, dtype: int64

**Колонка `purpose`**

In [28]:
df['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  674
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          624
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

#### 3.4.1 Вывод
Исправили данные:

* `children` — количество детей в семье. Удалили небольшую часть данных с выбивающимися слишком большими значениями (20 детей). Возможно там поставили лишний 0, но так таких строк менее 1% от выборки, выбрали удалить их.
* `gender` — пол. Удалили одну строку со значением `'XNA'`.

Необходимо категоризировать данные для колонок:
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## 4. Добавление данных

### 4.1 Категоризация данных

#### 4.1.1 Доход <a id="4.1.1"></a>

In [29]:
# функция для категоризации данных о доходах
def total_income_category(row):
    income = row['total_income']
    if 0 <= income <= 30000:
        return 'E'
    if 30001 <= income <= 50000:
        return 'D'
    if 50001 <= income <= 200000:
        return 'C'
    if 200001 <= income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'

In [30]:
df['total_income_category'] = df.apply(total_income_category, axis=1)

#### 4.1.2 Цели кредита <a id="4.1.2"></a>

In [31]:
# функция для категоризации целей кредита
def purpose_category(row):
    purpose = row['purpose']
    if 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'

In [32]:
df['purpose'] = df.apply(purpose_category, axis=1)

In [33]:
sorted(df['purpose'].unique())

['операции с автомобилем',
 'операции с недвижимостью',
 'получение образования',
 'проведение свадьбы']

### 4.2 Словари

#### 4.2.1 Образование <a id="4.2.1"></a>

In [34]:
education = df.pivot_table(index=['education_id', 'education'], values='gender', aggfunc='count').reset_index()
education = education.drop(columns='gender', axis = 1)
education

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


#### 4.2.2 Семейное положение <a id="4.2.2"></a>

In [35]:
family_status = df.pivot_table(index=['family_status_id', 'family_status'], values='gender', aggfunc='count').reset_index()
family_status = family_status.loc[family_status['gender'] != 0]
family_status = family_status.drop(columns='gender', axis = 1)
family_status

,family_status_id,family_status
4,0,женат / замужем
8,1,гражданский брак
12,2,вдовец / вдова
16,3,в разводе
20,4,Не женат / не замужем


#### 4.2.3 Доход <a id="4.2.3"></a>

In [36]:
total_income = pd.DataFrame(sorted(df['total_income_category'].unique()))
total_income['total_income'] = [
    'до 30000',
    '30001-50000',
    '50001-200000',
    '200001-1000000',
    'от 1000001'
]
total_income.columns = ['total_income_category', 'total_income']
total_income

,total_income_category,total_income
0,A,до 30000
1,B,30001-50000
2,C,50001-200000
3,D,200001-1000000
4,E,от 1000001


**Удаление столбцов из датафрейма**

In [37]:
df = df.drop(columns = ['education', 'family_status', 'total_income'],axis = 1)

### 4.3 Вывод

Для оптимизации работы с датафреймом:

1. Создали 5 категорий дохода.
2. Оптимизировали данные о цели получения кредита и объединили их в 4 категории.
3. Создали словари с данными об образовании, семейном положении и доходе.
4. Удалили данные об образовании, семейном положении и доходе из основного датафрейма, оставив только идентификаторы/категории.

## 5. Анализ данных

### 5.1 Зависимость между количеством детей и возвратом кредита в срок <a id="5.1"></a>

In [38]:
children_debt = df.pivot_table(index='children', columns='debt', values='gender', aggfunc='count', margins=True)
children_debt['no_debt_perc'] = (children_debt[0] / children_debt['All'] * 100).round(1)
children_debt['debt_perc'] = (children_debt[1] / children_debt['All'] * 100).round(1)
children_debt = children_debt.query('index != "All"').drop(columns='All', axis = 1)
children_debt

debt,0,1,no_debt_perc,debt_perc
children,,,,
0,13043.0,1063.0,92.5,7.5
1,4411.0,445.0,90.8,9.2
2,1858.0,194.0,90.5,9.5
3,303.0,27.0,91.8,8.2
4,37.0,4.0,90.2,9.8
5,9.0,NaN,100.0,NaN


#### 5.1.1 Вывод
Люди без детей с большей вероятностью не имеют задолженность по кредиту.

При этом из-за небольшого количества данных по категориям людей с 3, 4 и 5 детьми полученные результаты можно считать статистически недостоверными.

### 5.2 Зависимость между семейным положением и возвратом кредита в срок <a id="5.2"></a>

In [39]:
family_debt = df.pivot_table(index='family_status_id', columns='debt', values='gender', aggfunc='count', margins=True).reset_index()
family_debt['no_debt_perc'] = (family_debt[0] / family_debt['All'] * 100).round(1)
family_debt['debt_perc'] = (family_debt[1] / family_debt['All'] * 100).round(1)
family_debt = family_debt.query('index != 5').drop(columns='All', axis = 1)
family_debt.merge(family_status, on='family_status_id')

,family_status_id,0,1,no_debt_perc,debt_perc,family_status
0,0,11367,928,92.5,7.5,женат / замужем
1,1,3765,385,90.7,9.3,гражданский брак
2,2,892,63,93.4,6.6,вдовец / вдова
3,3,1109,84,93.0,7.0,в разводе
4,4,2528,273,90.3,9.7,Не женат / не замужем


#### 5.2.1 Вывод
Люди со статусом *'женат / замужем'*, *'вдовец / вдова'* и *'в разводе'* с большей вероятностью не имеют задолженность по кредиту.

Наибольшую вероятность не заплатить вовремя по кредиту у людей, которые не были в браке — со статусом *'гражданский брак'* и *'Не женат / не замужем'*.

При этом из-за небольшого количества данных по людям по статусом *'вдовец / вдова'* и *'в разводе'* полученные результаты можно считать статистически недостоверными.

### 5.3 Зависимость между уровнем дохода и возвратом кредита в срок <a id="5.3"></a>

In [40]:
income_debt = df.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count', margins=True)
income_debt['no_debt_perc'] = (income_debt[0] / income_debt['All'] * 100).round(1)
income_debt['debt_perc'] = (income_debt[1] / income_debt['All'] * 100).round(1)
income_debt = income_debt.query('index != "All"').drop(columns='All', axis = 1)
income_debt.merge(total_income, on='total_income_category')

,total_income_category,0,1,no_debt_perc,debt_perc,total_income
0,A,23,2,92.0,8.0,до 30000
1,B,4667,354,92.9,7.1,30001-50000
2,C,14622,1354,91.5,8.5,50001-200000
3,D,329,21,94.0,6.0,200001-1000000
4,E,20,2,90.9,9.1,от 1000001


#### 5.3.1 Вывод

При этом из-за небольшого количества данных по категориям A, D и E можно считать статистически достоверными данные только по категориям B и C.

В таком случае, люди с доходом  30001-50000 с большей вероятностью не имеют задолженность по кредиту.

### 5.4 Влияние целей кредита на его возврат в срок <a id="5.4"></a>

In [41]:
purpose_debt = df.pivot_table(index='purpose', columns='debt', values='gender', aggfunc='count', margins=True)
purpose_debt['no_debt_perc'] = (purpose_debt[0] / purpose_debt['All'] * 100).round(1)
purpose_debt['debt_perc'] = (purpose_debt[1] / purpose_debt['All'] * 100).round(1)
purpose_debt = purpose_debt.query('index != "All"').drop(columns='All', axis = 1)
purpose_debt

debt,0,1,no_debt_perc,debt_perc
purpose,,,,
операции с автомобилем,3891,401,90.7,9.3
операции с недвижимостью,9997,780,92.8,7.2
получение образования,3630,369,90.8,9.2
проведение свадьбы,2143,183,92.1,7.9


#### 5.4.1 Вывод

Люди, которые берут кредит на недвижимость или свадьбу с большей вероятностью не имеют задолженность по кредиту.

## 6. Общий вывод <a id="6"></a>

Первичный анализ показывает, что для полноценного анализа нужно больше данных.

Предварительно можно выделить **портрет клиента, который будет платить вовремя** — люди в браке, без детей, с доходом 30001-50000, которые берут кредит на недвижимость.

И **клиента, который имеет наибольшую вероятностью задолженности по кредиту** — люди не в браке (со статусом *'гражданский брак'* и *'Не женат / не замужем'*),  с 2 детьми, с доходом 50001-200000, которые берут кредит на автомобиль или образование.